In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models , layers
import tensorflow_hub as hub
import cv2

In [ ]:
img_width = 220
img_height = 220
batch_size = 32

In [ ]:
train_gen = ImageDataGenerator(
    rescale = 1./255 , 
    dtype=tf.float32 ,

)

In [ ]:
train_data = train_gen.flow_from_directory(
     "../input/messy-vs-clean-room/images/train" ,
    target_size = (img_height , img_width) ,
    color_mode = "rgb" ,
    class_mode='categorical',
    batch_size=batch_size, 
    shuffle=True,
    
)

In [ ]:
valid_gen = ImageDataGenerator(
    rescale = 1./255 , 
)

In [ ]:
valid_data = valid_gen.flow_from_directory(
    "../input/messy-vs-clean-room/images/val" ,
    target_size = (img_height , img_width) ,
    color_mode = "rgb" ,
    class_mode='categorical',
    batch_size=batch_size, 
    shuffle=True,
    
)

In [ ]:
from tensorflow.keras.applications import VGG16

In [ ]:
VGG16=VGG16(include_top=False,weights='imagenet',input_shape=(img_height,img_width,3))

In [ ]:
VGG16.trainable=False

In [ ]:
from keras.layers import Dense ,Flatten ,Conv2D ,MaxPooling2D ,Dropout ,BatchNormalization

In [ ]:
model = keras.Sequential(
[
    VGG16,
    MaxPooling2D(2,2) ,
    Flatten() , 
    Dense(280 , activation ="relu") ,
    Dropout(0.2) , 
    Dense(2 , activation ="softmax")
    
]
)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = "adam" , loss ="binary_crossentropy" , metrics = ["accuracy"])

In [ ]:
from tensorflow.keras.callbacks import Callback, EarlyStopping

In [ ]:
my_callbacks  = [EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=2,
                              mode='auto')]

In [ ]:
history = model.fit(train_data , validation_data = valid_data , epochs = 20 , callbacks = my_callbacks )

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
plt.title('Training accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training loss')
plt.legend()

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing import image as image_utils
from tensorflow.keras.applications.imagenet_utils import preprocess_input

def show_image(image_path):
    image = mpimg.imread(image_path)
    plt.imshow(image)

def make_predictions(image_path):
    show_image(image_path)
    image = image_utils.load_img(image_path, target_size=(img_height, img_width))
    image = image_utils.img_to_array(image)
    image = image.reshape(1,img_height,img_width,3)
    image = preprocess_input(image)
    preds = model.predict(image)
    return preds

In [ ]:
class_names = list(train_data.class_indices.keys())

In [ ]:
class_names[np.argmax(make_predictions("../input/messy-vs-clean-room/images/test/7.png"))]